# Exploring Ebay Car Sales Data

In this project, we'll work with a dataset of used cars from *eBay Kleinanzeigen*, a classifieds section of the German eBay website.

The dataset was originally uploaded to [Kaggle](https://www.kaggle.com/orgesleka/used-cars-database/data). A few modifications of the original have been made:

* 50,000 data points were sampled from the original dataset, to ensure that code runs smoothly
* Dataset has been dirtied a bit to more closely resemble what one would expect from a scraped dataset as the version uploaded to Kaggle was cleaned to be easier to work with

The data dictionary provided with data is as follows:

* `dateCrawled` - When this ad was first crawled. All field-values are taken from this date.
* `name` - Name of the car.
* `seller` - Whether the seller is private or a dealer.
* `offerType` - The type of listing
* `price` - The price on the ad to sell the car
* `abtest` - Whether the listing is included in an A/B test
* `vehicleType` - The vehicle type
* `yearOfRegistration` - The year in which the car was first registered
* `gearbox` - The transmission type
* `powerPS` - The power of the car in PS
* `model` - The car model name
* `kilometer` - How many kilometers the car has driven
* `monthOfRegistration` - The month in which the car was first registered
* `fuelType` - What type of fuel the car uses
* `brand` - The brand of the car
* `notRepairedDamage` - If the car has a damage which is not yet repaired
* `dateCreated` - The date in which the eBay listing was created
* `nrOfPictures` - The number of pictures in the ad
* `postalCode` - The postal code for the location of the vehicle
* `lastSeenOnline` - When the crawler saw this ad last online


The **aim** of this project is to clean the data and analyze the included used car listings.

In [2]:
import pandas as pd
import numpy as np

autos = pd.read_csv('autos.csv', encoding='Latin-1')

**Initial Exploration of the Data**

From below, we see that the data contains 20 columns of information for each of 50,000 cars as expected per the description above. The data appears to have some missing values: `vehicleType`, `gearBox`, `model`, `fuelType`, and `notRepairedDamaged` containing the most missing values. Also worth noting that the columns `yearOfRegistration`, `powerPS`, `monthOfRegistration`, `nrOfPictures`, and `postalCode` are all stored as integers in the `autos` dataframe. All of the other columns of information are stored as strings.

In [3]:
print(autos.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

Let's look at the first 5 rows in our dataframe:

From below, we see that there are some German words used among the colums that we may wish to later convert to their English equivalents: `seller`, `gearbox`, and `notRepairedDamage`. Also noting that the `odometer` column should probably be converted to a numerical data type as it is only stored as a string because of the "km" appended to the numbers and same goes for the `price` column due to the dollar sign prepended. The `dateCrawled`, `dateCreated`, and `lastSeen` columns seem to contain a uniform format for storing dates and times in the format "date-month-day hour:minute:second".

In [4]:
autos.head(5)

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


**Cleaning Column Names**

From above, we can also see that the column names use camelcase instead of Python's preferred snakecase. We'll start by converting the column names based on the data dictionary to be more descriptive and in snake case:

In [5]:
initial_columns = autos.columns

In [6]:
print(initial_columns)

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')


In [7]:
new_columns = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'nr_of_pictures', 'postal_code',
       'last_seen']

In [8]:
autos.columns = new_columns

In [9]:
autos.head(5)

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


**Initial Exploration and Cleaning**

From below, we see that `seller` and `offer_type` each have 49,999 of the same values and only one entry that is different. Thse columns are likely candidates to drop for later.

In [10]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-05 16:57:05,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


As noted before, the `price` and `odometer` columns are numeric values stored as text so we'll now convert each column to a numeric dtype after removing non-numeric characters.

In [11]:
autos["price"] = autos["price"].str.replace("$", "")
autos["price"] = autos["price"].str.replace(",", "")
autos["price"] = autos["price"].astype(int)

In [12]:
autos["odometer"] = autos["odometer"].str.replace("km", "")
autos["odometer"] = autos["odometer"].str.replace(",", "")
autos["odometer"] = autos["odometer"].astype(int)


We will rename the `odometer` column of the dataframe to `odometer_km` since it is important to maintain the unit information in converting to a numeric dtype

In [13]:
autos.rename(columns={'odometer': 'odometer_km'}, inplace=True)

**Exploring the Odometer and Price Columns**

We'll now analyze the `odometer_km` and `price` columns and take the following steps:

* Analyze the columns using minimum and maximum values and look for any values that look unrealistically high or low (outliers) that we might want to remove
* We'll use:
 1. `Series.unique().shape` to see how many unique values
 2. `Series.describe()` to view min/max/median/mean etc
 3. `Series.value_counts()`
 4. Remove outliers after investigating the above

Looks like there are a lot of unique price values: 2357 such values

In [14]:
autos["price"].unique().shape

(2357,)

The `.apply(lambda x: format(x, 'f'))` bit is to suppress scientific notation in the display below.

From the `max` value of 99999999 and `min` value of 0, it is clear that the `price` column contains outliers that need to be removed.

In [15]:
autos["price"].describe().apply(lambda x: format(x,'f'))

count       50000.000000
mean         9840.043760
std        481104.380500
min             0.000000
25%          1100.000000
50%          2950.000000
75%          7200.000000
max      99999999.000000
Name: price, dtype: object

There are 1421 cars in our data set with a price of 0 that also happens to be the most common price values:

In [16]:
len(autos[autos['price'] == 0].index)

1421

In [17]:
autos['price'].value_counts().sort_index().head(20)

0     1421
1      156
2        3
3        1
5        2
8        1
9        1
10       7
11       2
12       3
13       2
14       1
15       2
17       3
18       1
20       4
25       5
29       1
30       7
35       1
Name: price, dtype: int64

In [18]:
autos['price'].value_counts().sort_index().tail(20)

197000      1
198000      1
220000      1
250000      1
259000      1
265000      1
295000      1
299000      1
345000      1
350000      1
999990      1
999999      2
1234566     1
1300000     1
3890000     1
10000000    1
11111111    2
12345678    3
27322222    1
99999999    1
Name: price, dtype: int64

From above, we see that there are a fairly high number of listings below $30. There are also a small number of listings with very high values.

Given that eBay is an auction site, it is plausible that the opening bid starts at 1 dollar, so we will keep all of the values above this but remove anything below 350,000 dollars since it seems that prices increase steadily up to that number and then jump to more unrealistic numbers.

In [19]:
autos = autos[autos['price'].between(1,351000)]

Verify that this worked as expected:

In [20]:
autos['price'].describe()

count     48565.000000
mean       5888.935591
std        9059.854754
min           1.000000
25%        1200.000000
50%        3000.000000
75%        7490.000000
max      350000.000000
Name: price, dtype: float64

Now, we will perform the same kind of analysis for the `odometer_km` column:

In [21]:
autos["odometer_km"].unique().shape

(13,)

In [22]:
autos["odometer_km"].describe()

count     48565.000000
mean     125770.101925
std       39788.636804
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

In [23]:
autos["odometer_km"].value_counts()

150000    31414
125000     5057
100000     2115
90000      1734
80000      1415
70000      1217
60000      1155
50000      1012
5000        836
40000       815
30000       780
20000       762
10000       253
Name: odometer_km, dtype: int64

From above, it appears that there are only 13 unique values between 5000 and 150000 km, with 150000 being the most common value. There definitely appear to be more high mileage vehicles in our dataset overall. It is also clear that we are dealing with rounded values, one for every 10000 km or so, possibly due to a preset constraint on the eBay site in entering an approximate mileage for the vehicle.

**Exploring the date columns**

Taking a look at the three date columns containing string information:

In [25]:
autos[['date_crawled', 'ad_created', 'last_seen']][0:5]

,date_crawled,ad_created,last_seen
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50


From above, we can see that the first 10 characters represent represent the day. We will extract these for the three columns above, use `Series.value_counts()` to generate a distribution, and then sort by the index. The `normalize=True` and `dropna=False` arguments display percentages instead of counts and includes missing values.

In [27]:
(autos['date_crawled']
 .str[:10]
 .value_counts(normalize=True, dropna=False)
 .sort_index())

2016-03-05    0.025327
2016-03-06    0.014043
2016-03-07    0.036014
2016-03-08    0.033296
2016-03-09    0.033090
2016-03-10    0.032184
2016-03-11    0.032575
2016-03-12    0.036920
2016-03-13    0.015670
2016-03-14    0.036549
2016-03-15    0.034284
2016-03-16    0.029610
2016-03-17    0.031628
2016-03-18    0.012911
2016-03-19    0.034778
2016-03-20    0.037887
2016-03-21    0.037373
2016-03-22    0.032987
2016-03-23    0.032225
2016-03-24    0.029342
2016-03-25    0.031607
2016-03-26    0.032204
2016-03-27    0.031092
2016-03-28    0.034860
2016-03-29    0.034099
2016-03-30    0.033687
2016-03-31    0.031834
2016-04-01    0.033687
2016-04-02    0.035478
2016-04-03    0.038608
2016-04-04    0.036487
2016-04-05    0.013096
2016-04-06    0.003171
2016-04-07    0.001400
Name: date_crawled, dtype: float64

From above, it appears that the `date_crawled` column is pretty uniformly distributed between March 5th and April 7th in 2016

In [28]:
(autos['ad_created']
 .str[:10]
 .value_counts(normalize=True, dropna=False)
 .sort_index())

2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
2015-12-30    0.000021
2016-01-03    0.000021
2016-01-07    0.000021
2016-01-10    0.000041
2016-01-13    0.000021
2016-01-14    0.000021
2016-01-16    0.000021
2016-01-22    0.000021
2016-01-27    0.000062
2016-01-29    0.000021
2016-02-01    0.000021
2016-02-02    0.000041
2016-02-05    0.000041
2016-02-07    0.000021
2016-02-08    0.000021
2016-02-09    0.000021
2016-02-11    0.000021
2016-02-12    0.000041
2016-02-14    0.000041
2016-02-16    0.000021
2016-02-17    0.000021
2016-02-18    0.000041
2016-02-19    0.000062
2016-02-20    0.000041
2016-02-21    0.000062
                ...   
2016-03-09    0.033151
2016-03-10    0.031895
2016-03-11    0.032904
2016-03-12    0.036755
2016-03-13    0.017008
2016-03-14    0.035190
2016-03-15    0.034016
2016-03-16    0.030125
2016-03-17    0.031278
2016-03-18    0.013590
2016-03-19    0.033687
2016-03-20    0.037949
2016-03-21 

From above, it appears that the `ad_created` column spans a much wider range from June 11th, 2015 until April 7th, 2016.

In [29]:
(autos['last_seen']
 .str[:10]
 .value_counts(normalize=True, dropna=False)
 .sort_index())

2016-03-05    0.001071
2016-03-06    0.004324
2016-03-07    0.005395
2016-03-08    0.007413
2016-03-09    0.009595
2016-03-10    0.010666
2016-03-11    0.012375
2016-03-12    0.023783
2016-03-13    0.008895
2016-03-14    0.012602
2016-03-15    0.015876
2016-03-16    0.016452
2016-03-17    0.028086
2016-03-18    0.007351
2016-03-19    0.015834
2016-03-20    0.020653
2016-03-21    0.020632
2016-03-22    0.021373
2016-03-23    0.018532
2016-03-24    0.019767
2016-03-25    0.019211
2016-03-26    0.016802
2016-03-27    0.015649
2016-03-28    0.020859
2016-03-29    0.022341
2016-03-30    0.024771
2016-03-31    0.023783
2016-04-01    0.022794
2016-04-02    0.024915
2016-04-03    0.025203
2016-04-04    0.024483
2016-04-05    0.124761
2016-04-06    0.221806
2016-04-07    0.131947
Name: last_seen, dtype: float64

From above, `last_seen` appears to span a similar range of dates as `date_crawled`. Here, the crawler records the last date it saw any listing, which allows us to determine on what day the listing was removed, presumably due to the car being sold.

Worth noting here that the last three days in the `last_seen` column contains a disproportionate amount of last seen values. It seems unlikely that there was a huge spike in sales on those three days, but more likely that the crawling period ended.

Let's now look at the distribution of the `registration_year` column:

In [30]:
autos['registration_year'].describe()

count    48565.000000
mean      2004.755421
std         88.643887
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

In [32]:
autos['registration_year'].value_counts().sort_index()

1000       1
1001       1
1111       1
1800       2
1910       5
1927       1
1929       1
1931       1
1934       2
1937       4
1938       1
1939       1
1941       2
1943       1
1948       1
1950       3
1951       2
1952       1
1953       1
1954       2
1955       2
1956       4
1957       2
1958       4
1959       6
1960      23
1961       6
1962       4
1963       8
1964      12
        ... 
2000    3156
2001    2636
2002    2486
2003    2699
2004    2703
2005    2936
2006    2670
2007    2277
2008    2215
2009    2085
2010    1589
2011    1623
2012    1310
2013     803
2014     663
2015     392
2016    1220
2017    1392
2018     470
2019       2
2800       1
4100       1
4500       1
4800       1
5000       4
5911       1
6200       1
8888       1
9000       1
9999       3
Name: registration_year, Length: 95, dtype: int64

From above, it is clear that there is some incorrect input as the dates should not go above the year 2016 or below 1927.

**Dealing with Incorrect Registration Year Data**

One thing that stands out from the exploration above is that the `registration_year` column has a minimum value of 1000, long before cars were even invented and a maximum value of 9999, many years into the future.


Because a car can't be registered before after a listing was seen, any vehicle with a registration year above 2016 is definitely inaccurate.

Let's count the number of listings with cars that fall outside the 1900 to 2016 interval to see if it's safe to remove these rows entirely, or if we need more custom logic.

In [40]:
len(autos[~autos['registration_year'].between(1900,2016)].index)

1884

From above we see that there are 1884 registration years that fall outside of the 1900-2016 interval, which is less than 4% of the total values in our dataset so it is probably safe to drop these values.

In [43]:
autos = autos[autos['registration_year'].between(1900, 2016)]

In [46]:
autos['registration_year'].value_counts(normalize=True).head(10)

2000    0.067608
2005    0.062895
1999    0.062060
2004    0.057904
2003    0.057818
2006    0.057197
2001    0.056468
2002    0.053255
1998    0.050620
2007    0.048778
Name: registration_year, dtype: float64

It appears that most vehicles were first registered in the past 20 years.

**Exploring Price by Brand**

Here we will explore the unique values in the `brand` column, and decide on which brands we want to aggregate on.

In [52]:
autos['brand'].value_counts(normalize=True)

volkswagen        0.211264
bmw               0.110045
opel              0.107581
mercedes_benz     0.096463
audi              0.086566
ford              0.069900
renault           0.047150
peugeot           0.029841
fiat              0.025642
seat              0.018273
skoda             0.016409
nissan            0.015274
mazda             0.015188
smart             0.014160
citroen           0.014010
toyota            0.012703
hyundai           0.010025
sonstige_autos    0.009811
volvo             0.009147
mini              0.008762
mitsubishi        0.008226
honda             0.007840
kia               0.007069
alfa_romeo        0.006641
porsche           0.006127
suzuki            0.005934
chevrolet         0.005698
chrysler          0.003513
dacia             0.002635
daihatsu          0.002506
jeep              0.002271
subaru            0.002142
land_rover        0.002099
saab              0.001649
jaguar            0.001564
daewoo            0.001500
trabant           0.001392
r

From above, there are lots of brands that don't have too many listings so we will limit our analysis to those brands that comprise more than 5% of the total listings.

In [53]:
brand_counts = autos['brand'].value_counts(normalize=True)

In [56]:
common_brands = brand_counts[brand_counts > 0.05].index

In [58]:
price_by_brand = {}

In [61]:
for c in common_brands:
    selected_rows = autos[autos['brand'] == c]
    mean_price = selected_rows['price'].mean()
    price_by_brand[c] = int(mean_price)

print(price_by_brand)

{'volkswagen': 5402, 'ford': 3749, 'bmw': 8332, 'opel': 2975, 'audi': 9336, 'mercedes_benz': 8628}


From above, there is a distinct price gap: Audi, BMW, and Mercedes-Benz are the most expensive. Ford and Opel are less expensive with Volkswagen in the middle. The could explain the popularity of Volkswagen as it is the 'best of both worlds' option here.

We will now aggregate on the mean mileage for each of the top six brands from above and store the results in a dictionary.

In [64]:
mileage_by_brand = {}
for c in common_brands:
    selected_rows = autos[autos['brand'] == c]
    mean_mileage = selected_rows['odometer_km'].mean()
  
    mileage_by_brand[c] = int(mean_mileage)
    
print(mileage_by_brand)

    

{'volkswagen': 128707, 'ford': 124266, 'bmw': 132572, 'opel': 129310, 'audi': 129157, 'mercedes_benz': 130788}


In [69]:
bmp_series = pd.Series(price_by_brand)
bmm_series = pd.Series(mileage_by_brand)


In [70]:
top_brands = pd.DataFrame(bmp_series, columns=['mean_price'])

In [72]:
top_brands['mean_mileage'] = bmm_series

In [73]:
top_brands

,mean_price,mean_mileage
audi,9336,129157
bmw,8332,132572
ford,3749,124266
mercedes_benz,8628,130788
opel,2975,129310
volkswagen,5402,128707


From above, The range of car mileages does not vary as much as the prices do by brand. There is a slight trend to the more expensive vehicles having higher mileage, with the less expensive vehicles having lower mileage.